In [114]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import pivot_table
import json

In [ ]:
### Build dataset for UK Hours Worked application
### application can be found at https://rangewell-hours.herokuapp.com/
### raw data is from ONS

In [115]:
pd.set_option('display.width', 1000)

In [116]:
horig = pd.read_csv("hours_full.csv", header=0)

print(horig.shape)
horig.head(2)

C:\Users\liamk\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(836244, 15)


,V4_2,Data marking,Coefficient of variation,Time_codelist,Time,ashe-geography,Geography,Hours_codelist,Hours,Sex_codelist,Sex,WorkingPattern_codelist,WorkingPattern,Statistics_codelist,Statistics
0,15.1,NaN,0.5,Year,2012,K02000001,United Kingdom,paid-hours-worked-total,Paid hours worked - Total,all,All,all,All,10th-percentile,10th percentile
1,15.1,NaN,0.6,Year,2012,K03000001,Great Britain,paid-hours-worked-total,Paid hours worked - Total,all,All,all,All,10th-percentile,10th percentile


In [117]:
print(f"Unique IDs: {len(horig['ashe-geography'].unique())}")
print(horig.columns)

Unique IDs: 431
Index(['V4_2', 'Data marking', 'Coefficient of variation', 'Time_codelist', 'Time', 'ashe-geography', 'Geography', 'Hours_codelist', 'Hours', 'Sex_codelist', 'Sex', 'WorkingPattern_codelist', 'WorkingPattern', 'Statistics_codelist', 'Statistics'], dtype='object')


In [118]:
horig['WorkingPattern'].unique()

array(['All', 'Full-Time', 'Part-Time'], dtype=object)

In [119]:
hedit = horig.copy()

hedit = hedit[hedit['Hours'] != 'Paid hours worked - Total']
hedit = hedit[hedit['WorkingPattern'] != 'All']
hedit = hedit[hedit['Statistics'] == 'Mean']

hedit = hedit[['ashe-geography','Geography','Time','Sex','Hours','WorkingPattern','V4_2','Coefficient of variation']]
hedit.rename(columns={"ashe-geography":"ID", "Time":"Year","V4_2":"Value","Coefficient of variation":"CV"}, inplace=True)
hedit.reset_index(drop=True, inplace=True)

print(hedit.shape)
hedit.head()

(30960, 8)


,ID,Geography,Year,Sex,Hours,WorkingPattern,Value,CV
0,K02000001,United Kingdom,2012,All,Paid hours worked - Basic,Full-Time,37.9,0
1,K03000001,Great Britain,2012,All,Paid hours worked - Basic,Full-Time,37.9,0
2,K04000001,England and Wales,2012,All,Paid hours worked - Basic,Full-Time,37.9,0
3,E92000001,England,2012,All,Paid hours worked - Basic,Full-Time,37.9,0
4,E12000001,North East,2012,All,Paid hours worked - Basic,Full-Time,37.8,0.2


In [120]:
hedit.loc[hedit.Hours == 'Paid hours worked - Basic', 'Hours'] = 'Basic'
hedit.loc[hedit.Hours == 'Paid hours worked - Overtime', 'Hours'] = 'Overtime'
hedit.loc[hedit.WorkingPattern == 'Full-Time', 'WorkingPattern'] = 'Full'
hedit.loc[hedit.WorkingPattern == 'Part-Time', 'WorkingPattern'] = 'Part'

hedit.head(2)

,ID,Geography,Year,Sex,Hours,WorkingPattern,Value,CV
0,K02000001,United Kingdom,2012,All,Basic,Full,37.9,0
1,K03000001,Great Britain,2012,All,Basic,Full,37.9,0


In [121]:
# mismatch from geojson
# also "N09..." N Ireland removed due to lack of data
hedit.loc[hedit.Geography == 'Fife', 'ID'] = 'S12000047'
hedit.loc[hedit.Geography == 'Perth and Kinross', 'ID'] = 'S12000048'

## Append Lat Lon

In [122]:
geo = pd.read_csv('Local_Authority_Districts_December_2018_Ultra_Generalised_Clipped_Boundaries_UK.csv')

print(geo.shape)
geo.head(2)

(391, 10)


,objectid,lad18cd,lad18nm,lad18nmw,bng_e,bng_n,long,lat,st_lengthshape,st_areashape
0,1,E06000001,Hartlepool,,447157,531476,-1.27023,54.676201,50488.387081,9.651231e+07
1,2,E06000002,Middlesbrough,,451141,516887,-1.21099,54.544701,35506.404054,5.522915e+07


In [123]:
geo = geo[['lad18cd', 'lat', 'long']]
geo.rename(columns={"lad18cd":"ID", "long":"lon"}, inplace=True)

geo.head(2)

,ID,lat,lon
0,E06000001,54.676201,-1.27023
1,E06000002,54.544701,-1.21099


In [124]:
# rows removed are N Ireland + Country / Region rows
hla = hedit.copy()

print(hla.shape)

hla = pd.merge(hla, geo, on='ID')

print(hla.shape)
print(len(hla.ID.unique()))
hla.head(2)

(30960, 8)
(27288, 10)
380


,ID,Geography,Year,Sex,Hours,WorkingPattern,Value,CV,lat,lon
0,E06000005,Darlington,2012,All,Basic,Full,38.2,0.9,54.535301,-1.56835
1,E06000005,Darlington,2012,All,Basic,Part,17.6,4.4,54.535301,-1.56835


In [129]:
hla.describe()

,Year,lat,lon
count,27288.000000,27288.000000,27288.000000
mean,2014.500733,52.621511,-1.482286
std,1.707813,1.617132,1.491451
min,2012.000000,49.923302,-6.657220
25%,2013.000000,51.464401,-2.561670
50%,2015.000000,52.237801,-1.373440
75%,2016.000000,53.416698,-0.275680
max,2017.000000,60.505001,1.649500


In [128]:
# print(hla.shape)
# hla.to_csv('h_clean_all.csv')

# h2017 = hla[hla.Year == 2017]
# print(h2017.shape)

# h2017.to_csv('h_clean_2017.csv')

(27288, 10)
(4550, 10)


In [49]:
hnew = pd.read_csv("h_clean_2017.csv", header=0, index_col=0)

print(hnew.shape)
hnew.head(2)

(5162, 8)


,ID,Geography,Year,Sex,Hours,WorkingPattern,Value,CV
25798,W06000020,Torfaen,2017,All,Basic,Full,38.7,1.4
25799,W06000021,Monmouthshire,2017,All,Basic,Full,38.4,1


In [50]:
hnew.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5162 entries, 25798 to 30959
Data columns (total 8 columns):
ID                5162 non-null object
Geography         5162 non-null object
Year              5162 non-null int64
Sex               5162 non-null object
Hours             5162 non-null object
WorkingPattern    5162 non-null object
Value             3148 non-null float64
CV                5162 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 363.0+ KB


In [53]:
hbasicfull = hnew[(hnew.Sex == 'All') & (hnew.Hours == 'Basic') & (hnew.WorkingPattern == 'Full')]

print(f"Unique values: {len(hbasicfull.Value.unique())}")
hbasicfull.Value.describe()

Unique values: 39


count    431.000000
mean      38.268445
std        0.665933
min       36.500000
25%       37.800000
50%       38.200000
75%       38.600000
max       41.600000
Name: Value, dtype: float64

## Cross Verify with geojson

In [58]:
la = pd.read_csv('Local_Authority_Districts_December_2018_Ultra_Generalised_Clipped_Boundaries_UK.csv')

print(la.shape)
la.head(2)

(391, 10)


,objectid,lad18cd,lad18nm,lad18nmw,bng_e,bng_n,long,lat,st_lengthshape,st_areashape
0,1,E06000001,Hartlepool,,447157,531476,-1.27023,54.676201,50488.387081,9.651231e+07
1,2,E06000002,Middlesbrough,,451141,516887,-1.21099,54.544701,35506.404054,5.522915e+07


In [67]:
new_id = hnew.ID.unique().tolist()
geo_id = la.lad18cd.unique().tolist()
print(len(new_id))
print(len(geo_id))

431
391


In [70]:
# these need to be added to Hours Worked df
diff_1 = [item for item in geo_id if item not in new_id]

print(len(diff_1))
diff_1

13


['N09000001',
 'N09000002',
 'N09000003',
 'N09000004',
 'N09000005',
 'N09000006',
 'N09000007',
 'N09000008',
 'N09000009',
 'N09000010',
 'N09000011',
 'S12000047',
 'S12000048']

In [102]:
la[la.lad18cd == 'S12000047']

,objectid,lad18cd,lad18nm,lad18nmw,bng_e,bng_n,long,lat,st_lengthshape,st_areashape
367,368,S12000047,Fife,,339197,704726,-2.98235,56.231201,250355.120713,1.326132e+09


In [75]:
# These are countries and regions; need separate geojson
diff_2 = [item for item in new_id if item not in geo_id]

print(len(diff_2))
diff_2[:10]

53


['S92000003',
 'K02000001',
 'K03000001',
 'K04000001',
 'E92000001',
 'E12000001',
 'E11000007',
 'E12000002',
 'E10000006',
 'E11000001']

In [90]:
diff_2[-3]

'S12000015'

In [96]:
hnew[hnew.Geography == 'Fife']

,ID,Geography,Year,Sex,Hours,WorkingPattern,Value,CV
26209,S12000015,Fife,2017,All,Basic,Full,37.5,0.5
26639,S12000015,Fife,2017,All,Basic,Part,18.4,2.7
27070,S12000015,Fife,2017,Male,Basic,Full,38.1,0.5
27499,S12000015,Fife,2017,Male,Basic,Part,18.6,7.1
27929,S12000015,Fife,2017,Female,Basic,Full,36.7,1
28359,S12000015,Fife,2017,Female,Basic,Part,18.4,2.8
28802,S12000015,Fife,2017,All,Overtime,Full,1.2,12
29220,S12000015,Fife,2017,All,Overtime,Part,NaN,25
29651,S12000015,Fife,2017,Male,Overtime,Full,1.5,14
30080,S12000015,Fife,2017,Male,Overtime,Part,NaN,41


In [36]:
# pv = pivot_table(hedit, values='V4_2', index=['Time', 'Hours'], 
#                  columns=['Sex'], aggfunc=np.mean)
# pv